In [1]:
source("https://raw.githubusercontent.com/balthapaixao/Covid19_BR_underreport/master/Func_/new_preproc.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape’


The following object is masked from ‘package:dplyr’:

    rename


Loading required package: greybox

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Package "greybox", v0.6.0 loaded.

This is package "smooth", v2.6.0

Loading required package: forecast


Attaching package: ‘ModelMetrics’


The following object is masked from ‘package:base’:

    kappa


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: Rcpp


Attaching package: ‘RSNNS’


The following object is masked from ‘package:ModelMetrics’:

    confusionMatrix


Loading required package: KernelKn

# Preparing the data

In [2]:
#X <- srag #recent data acquired from Infogripe Gitlab
X <- read_csv('https://raw.githubusercontent.com/balthapaixao/Covid19_BR_underreport/master/Aux_arqs/srag_data_28_05.csv') #data used for the article

Parsed with column specification:
cols(
  .default = col_double(),
  `data de publicação` = col_date(format = ""),
  `Unidade da Federação` = col_character(),
  Tipo = col_character(),
  dado = col_character(),
  escala = col_character(),
  sexo = col_character(),
  semana = col_character(),
  `Ano e semana epidemiológica` = col_character(),
  `Situação do dado` = col_character(),
  `0-4 anos` = col_logical()
)

See spec(...) for full column specifications.



In [3]:
date = '2020-05-02'
#For choosing the date the format is 'YYYY-mm-dd',
#default date is 2020-05-02 - 18th Epidemiological Week
# the date must be the end of an EW 
#https://portalsinan.saude.gov.br/calendario-epidemiologico-2020

data <- pre_proc_data(X, "underreport", date) #Preparing the data for underreport analysis
hm_data <- pre_proc_ms(date)

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_double(),
  Sigla = col_character(),
  Codigo = col_double(),
  Estado = col_character()
)

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_double(),
  Sigla = col_character(),
  Codigo = col_double(),
  Estado = col_character()
)



## After pre_proc_data("underreport") the data can be acessed by

<a href="https://github.com/balthapaixao/Covid19_BR_underreport/blob/master/Aux_arqs/dataflow_covid.png"><img src="https://i.ibb.co/hLwQJkp/dataflow-covid.png" alt="dataflow-covid" border="0"></a><br /><a target='_blank' href='https://github.com/balthapaixao/Covid19_BR_underreport/blob/master/Aux_arqs/dataflow_covid.png'>Dataflow Image </a><br />

In [12]:
#selecting cases series
serie <- data$deaths$no_covid
serie_covid <- data$deaths$covid

hm <- hm_data$hm_acc_deaths #Health Ministry data for rate_table

In [5]:
error_df <- calc_error(serie)
error_df

[1] "------- GENERATING THE ERRORS OF THE MODEL TABLE -------"


,epsilon_baseline,interval
,<dbl>,<fct>
AC,1.727,"[1.177, 2.305]"
AL,1.482,"[0.972, 2.028]"
AM,9.770,"[6.689, 14.797]"
AP,0.299,"[0.163, 0.457]"
BA,10.211,"[7.582, 13.304]"
CE,6.967,"[4.397, 10.813]"
DF,13.036,"[11.223, 14.998]"
ES,4.021,"[2.853, 5.709]"
GO,6.349,"[3.413, 10.248]"


In [14]:
rate_table <- calc_underreport(serie, serie_covid, hm)

[1] "------- GENERATING THE CUMULATIVE UNDERREPORTED TABLE -------"


Warning message in wilcox.test.default(dots[[1L]][[22L]], dots[[2L]][[22L]], alternative = dots[[3L]][[1L]], :
“cannot compute exact p-value with ties”


[1] "------- P-VALUES TEST TABLE -------"
   random_noise_test reported_values_test
AC             FALSE                FALSE
AL              TRUE                 TRUE
AM              TRUE                 TRUE
AP              TRUE                FALSE
BA              TRUE                 TRUE
CE              TRUE                 TRUE
DF              TRUE                 TRUE
ES              TRUE                FALSE
GO              TRUE                FALSE
MA              TRUE                 TRUE
MG              TRUE                 TRUE
MS             FALSE                FALSE
MT              TRUE                FALSE
PA              TRUE                 TRUE
PB              TRUE                 TRUE
PE              TRUE                 TRUE
PI              TRUE                 TRUE
PR              TRUE                 TRUE
RJ              TRUE                 TRUE
RN              TRUE                 TRUE
RO             FALSE                FALSE
RR              TRUE              

In [15]:
print("------- GENERATING THE UNDER-REPORTING RATES TABLE -------")
rate_table

[1] "------- GENERATING THE UNDER-REPORTING RATES TABLE -------"


,Cum._Novelty_SARI,Cum._Covid_SARI,Rate,Cum._Covid_HM
,<dbl>,<dbl>,<chr>,<cst_mtrx>
AC,0,13,*,21
AL,49,34,0.441 +/- 0.029,58
AM,2023,1147,0.764 +/- 0.003,501
AP,26,21,**,40
BA,200,110,0.818 +/- 0.027,123
CE,1429,983,0.454 +/- 0.004,614
DF,90,33,1.727 +/- 0.061,31
ES,91,84,**,102
GO,61,36,**,30
